# Copy Files with No Duplicates!

## Instructions

To use this notebook, run all cells in order, up to and including Python cell 15, which runs the main function.

### Deduplicating a Directory

Suppose I have some directory, `Directory 2022`.

I want to create a copy of that directory which contains no duplicates.
Note that here, duplicates mean files which are exactly the same content-wise (file name, age, and location are not taken into consideration).

1. Run all cells, and select **COPY** mode
2. Enter a hash table file, e.g. "MyDirectoryHashed.json"
3. Select directory 2022 as your source (e.g. `C:\My Files\Directory 2022`)
4. Select a new, empty directory as your destination (e.g. `C:\My Files\Deduplicated Directory`)
5. (Optional) Enter *YES* to store additional info in the hash table file

The deduplicated directory will contain all files in directory 2022, minus duplicates.
In the case of a duplicate, only the oldest file will be kept.
All file names (and additional info, if desired) will be kept in the hash table file.

The source directory will be untouched.

### Merging a Directory into an Already Deduplicated Directory

Suppose I have another directory, `Directory 2023`, which is a later version of the same directory from part 1.

It will have a lot of the same files, which we don't want to keep, but also some new files.

1. Run all cells, and select **COPY** mode
2. Enter the existing hash table file created in part 1, e.g. "MyDirectoryHashed.json"
3. Select directory 2023 as your source (e.g. `C:\My Files\Directory 2023`)
4. Select the directory created in part 1 as your destination (e.g. `C:\My Files\Deduplicated Directory`)
5. (Optional) Enter *YES* to store additional info in the hash table file; if you did this in part 1, it makes sense to do it again for consistency

The deduplicated directory will now contain all files in directory 2022 and directory 2023, minus duplicates.

The source directory will be untouched.

### Taking Note of Duplicates Without Moving Files

Suppose I have a directory, `My Stuff`, and I want to create a record of its contents and get a sense of how many duplicates there are.

1. Run all cells, and select **MARK** mode
2. Enter a hash table file, e.g. "MyStuffHashed.json"
3. Select the directory as your source (e.g. `C:\My Files\My Stuff`)
4. (Optional) Enter *YES* to store additional info in the hash table file

All file names (and additional info, if desired) will be kept in the hash table file.  No files will have been copied.


## Imports

This notebook only uses Python Standard Library imports.

In [ ]:
import hashlib
import json
import os
import shutil
import time
from pathlib import Path

## Serializing and Deserializing the Hash Table

In [ ]:
def load_hash_table(hash_table_path: str) -> dict:
    """Loads a hash table dict from a file."""

    try:
        with open(hash_table_path, 'r') as f_json:
            return json.load(f_json)
    except FileNotFoundError:
        return {}


def save_hash_table(hash_table_path: str, hash_table: dict):
    """Loads a hash table dict from a file."""
    
    DEFAULT = 'file_data.json'

    try:
        with open(hash_table_path, 'w') as f_json:
            json.dump(hash_table, f_json, indent=4, sort_keys=True)
    except FileNotFoundError:
        try:
            with open(DEFAULT, 'w') as f_json:
                json.dump(hash_table, f_json, indent=4, sort_keys=True)
                print(f'Could not create json file; used default name ({DEFAULT}) instead.')
        except FileNotFoundError:
            global unsaved_hash_table
            unsaved_hash_table = hash_table
            print('Could not create json file.')

## Hashing Files

In [ ]:
def create_md5(file_path: str) -> str:
    """Create an MD5 hash of a file's contents."""
    
    hash_md5 = hashlib.md5()
    try:
        with open(file_path, 'rb') as f_to_hash:
            for chunk in iter(lambda: f_to_hash.read(4096), b''):
                hash_md5.update(chunk)
    except FileNotFoundError:
        return 'BADHASH'
    
    return hash_md5.hexdigest()


def hash_file(hash_table: dict, file_path_object, dir_path: str = None, extra_details: bool = False ) -> bool:
    """Attempt to hash a file and return true iff it was not already hashed."""

    file_path = str(file_path_object)
    file_size = str(os.path.getsize(file_path))
    file_name = file_path_object.name
    file_hash = create_md5(file_path)
    combined_hash = f'{file_hash}{file_size}'
    
    if extra_details:
        relative_path = file_path.replace(dir_path, "") if dir_path else file_path
        
        if combined_hash in hash_table:
            if file_name not in hash_table[combined_hash]['names']:
                hash_table[combined_hash]['names'].append(file_name)
            if relative_path not in hash_table[combined_hash]['paths']:
                hash_table[combined_hash]['paths'].append(relative_path)
            return False
        else:
            hash_table[combined_hash] = {
                'md5': file_hash,
                'size': file_size,
                'names': [file_name],
                'paths': [relative_path],
                'created': get_earliest_age(file_path_object)
            }
            return True

    else:
        if combined_hash in hash_table:
            if file_name not in hash_table[combined_hash]['names']:
                hash_table[combined_hash]['names'].append(file_name)
            return False
        else:
            hash_table[combined_hash] = {
                'md5': file_hash,
                'size': file_size,
                'names': [file_name]
            }
            return True

## Moving Files

In [ ]:
def move_files_relative(old_dir_path: str, new_dir_path: str, file_path: str):
    """Moves a file from one directory to another, keeping its relative file structure."""

    if os.path.isfile(file_path):
        relative_path = file_path.replace(old_dir_path, "")
        new_path = f'{new_dir_path}{relative_path}'
        os.makedirs(os.sep.join(new_path.split(os.sep)[:-1]), exist_ok=True)
        shutil.copyfile(file_path, new_path)

## Iterating Through Files

In [ ]:
def get_earliest_age(file_path_object) -> int:
    """Returns the earliest of created and modified time for a file path object."""
    
    try:
        return min(
            os.path.getmtime(str(file_path_object)),
            os.path.getctime(str(file_path_object))
        )
    except FileNotFoundError:
        return 0

In [ ]:
def hash_files_in_directory(hash_table: dict, dir_path: str, extra_details: bool = False):
    """Hashes all files in a directory."""
    
    hashed = 0
    duplicate = 0
    
    file_paths = sorted(Path(dir_path).glob('**/*'), key=get_earliest_age)
    for file_path in file_paths:
        if os.path.isdir(str(file_path)):
            continue
        
        if hash_file(hash_table, file_path, dir_path, extra_details):
            hashed += 1
            print(f'Hashed: {file_path}')
        else:
            duplicate += 1
            print(f'Hashed, Duplicate: {file_path}')
    
    print(f'\n{hashed + duplicate} total files, {hashed} new hashes, {duplicate} duplicates')


def hash_and_copy_files_in_directory(hash_table: dict, old_dir_path: str, new_dir_path: str, extra_details: bool = False):
    """Hashes all files in a directory."""
    
    hashed = 0
    duplicate = 0
    failures = []
    
    file_paths = sorted(Path(old_dir_path).glob('**/*'), key=get_earliest_age)
    for file_path in file_paths:
        if os.path.isdir(str(file_path)):
            continue
        
        try:
            if hash_file(hash_table, file_path, old_dir_path, extra_details):
                move_files_relative(old_dir_path, new_dir_path, str(file_path))
                print(f'Hashed, Moved: {file_path}')
                hashed += 1
            else:
                print(f'Hashed, Duplicate: {file_path}')
                duplicate += 1
        except FileNotFoundError:
            print(f'FAILED: {file_path}')
            failures.append(str(file_path))
    
    fail_count = len(failures)
    print(f'\n{hashed + duplicate + fail_count} total files, {hashed} moved, {duplicate} duplicates')
    if failures:
        print(f'\n{fail_count} files failed:')
        for fail in failures:
            print(f'* {fail}')

## Run

In [ ]:
def mark_files():
    """Add files in a directory to a hash table."""
    
    hash_table_file = input('Hash table file: ')
    source_directory = input('Source directory: ')
    
    do_extra_details = input('Would you like to record earliest time of creation and all file paths? ')
    extra_details = do_extra_details.lower()[0] == 'y'
    
    print('\nProcessing...\n')
    
    hash_table = load_hash_table(hash_table_file)
    hash_files_in_directory(hash_table, source_directory, extra_details)
    save_hash_table(hash_table_file, hash_table)
    
    print('\nDone!')


def copy_files():
    """Hash and copy unhashed files in a directory to another folder."""

    hash_table_file = input('Hash table file: ')
    source_directory = input('Source directory: ')
    destination_directory = input('Destination directory: ')
    
    do_extra_details = input('Would you like to record earliest time of creation and all file paths? ')
    extra_details = do_extra_details.lower()[0] == 'y'
    
    print('\nProcessing...\n')
    
    hash_table = load_hash_table(hash_table_file)
    hash_and_copy_files_in_directory(hash_table, source_directory, destination_directory, extra_details)
    save_hash_table(hash_table_file, hash_table)
    
    print('\nDone!')

    
def debug():
    print('Hello worlds, we are in debug mode!  Stick code here to mess around with this notebook.')
    print('If this isn\'t what you wanted, just re-run this cell.')

Example directories:

```
D:\Dropbox\Project Hub\Game_and_Programming_Tutorials\Python File Manipulation\Armistice_HT.json
D:\Dropbox\Project Hub\Game_and_Programming_Tutorials\Python File Manipulation\Armistice
D:\Dropbox\Project Hub\Game_and_Programming_Tutorials\Python File Manipulation\Armistice_New

phone_backups.json
D:\Media\Phone Backups\LG G5
C:\Media\Phone Backups Temp\Deduplicated Phone Backups\LG G5
D:\Media\Phone Backups\From 32 GB SD Card
C:\Media\Phone Backups Temp\Note 9 2022-02-06
D:\Media\Phone Backups\Note 9\From SD Card 2020-11-09
C:\Media\Straggler Files 2021-03-27
```

In [ ]:
def main():
    print('Copy files with no duplicates!')
    print('For simplicity, please do not use relative file paths and do not include trailing slashes.')
    mode = input('Mode (MARK or COPY): ')
    print('')
    if mode.lower() == 'mark':
        confirm = input('Are you sure you want to do MARK mode, and not COPY? ')
        if confirm.lower()[0] == 'y':
            mark_files()
    elif mode.lower() == 'copy':
        copy_files()
    elif mode.lower() == 'debug':
        debug()
    else:
        print('No action taken!')


main()

## Additional Documentation

### Motivation

The goal of this notebook is to have two modes, such that multiple directories can be merged without duplicates within each directory or between any of them:

1. Mark only mode: Given a directory, hash each file using MD5 and file size
2. Copy mode: Same as mark mode, plus copy each file not in the hash table to another directory

This requires four pieces of functionality:

1. Serializing and deserializing a Python dict containing the data (hash and size)
2. Iterating through files
3. Hashing the file in question and extracting its size
4. Copying files from one directory to another while maintaining their relative directory structure

### Sources

* D:\Dropbox\Project Hub\Website\KieferFlaskSite\home\scripts\file_age_directory.py
* https://stackoverflow.com/questions/8858008/how-to-move-a-file-in-python#8858026
* https://stackoverflow.com/questions/1072569/see-if-two-files-have-the-same-content-in-python
* https://stackoverflow.com/questions/5787471/md5-and-sha-2-collisions-in-python
* https://stackoverflow.com/questions/3431825/generating-an-md5-checksum-of-a-file
* https://stackoverflow.com/questions/6773584/how-is-pythons-glob-glob-ordered